In [ ]:
imageSize=50
train_dir = "./SignTranslate/Dataset/train/"
test_dir =  "./SignTranslate/Dataset/test/"
import os
from tqdm import tqdm
def get_data(folder):
    """
    Load the data and labels from the given folder.
    """
    X = []
    y = []
    
    for folderName in os.listdir(folder):
        if not folderName.startswith('.'):
            for filename in os.listdir(folderName):
                if filename.startswith("A"):
                    label = 0
                elif filename.startswith('B'):
                    label = 1
                elif filename.startswith('C'):
                    label = 2
                elif filename.startswith('D'):
                    label = 3
                elif filename.startswith('E'):
                    label = 4
                elif filename.startswith('F'):
                    label = 5
                elif filename.startswith('G'):
                    label = 6
                elif filename.startswith('H'):
                    label = 7
                elif filename.startswith('I'):
                    label = 8
                elif filename.startswith('J'):
                    label = 9
                elif filename.startswith('K'):
                    label = 10
                elif filename.startswith('L'):
                    label = 11
                elif filename.startswith('M'):
                    label = 12
                elif filename.startswith('N'):
                    label = 13
                elif filename.startswith('O'):
                    label = 14
                elif filename.startswith('P'):
                    label = 15
                elif filename.startswith('Q'):
                    label = 16
                elif filename.startswith('R'):
                    label = 17
                elif filename.startswith('S'):
                    label = 18
                elif filename.startswith('T'):
                    label = 19
                elif filename.startswith('U'):
                    label = 20
                elif filename.startswith('V'):
                    label = 21
                elif filename.startswith('W'):
                    label = 22
                elif filename.startswith('X'):
                    label = 23
                elif filename.startswith('Y'):
                    label = 24
                elif filename.startswith('Z'):
                    label = 25

                img_file = cv2.imread(folder + folderName + '/' +filename)
                if img_file is not None:
                    img_arr = np.asarray(img_file)
                    X.append(img_arr)
                    y.append(label)
        X = np.asarray(X)
        y = np.asarray(y)
        return X,y
X_train, y_train = get_data(train_dir) 
X_test, y_test= get_data(test_dir) # Too few images

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2) 

# Encode labels to hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
#from keras.utils.np_utils import to_categorical
#y_trainHot = to_categorical(y_train, num_classes = 30)
#y_testHot = to_categorical(y_test, num_classes = 30)